In [10]:
mydata = open('SCC.txt','r').readlines()
print(len(mydata))
num_vertices = 875714

5105043


In [11]:
import sys ,resource
sys.setrecursionlimit(4000000)
resource.setrlimit(resource.RLIMIT_STACK, (2**29, 2**30))


In [12]:
!ulimit -s 65532

In [13]:
import numpy as np
import pandas as pd
def show_mem_usage():
    '''Displays memory usage from inspection
    of global variables in this notebook'''
    gl = sys._getframe(1).f_globals
    vars= {}
    for k,v in list(gl.items()):
        # for pandas dataframes
        if hasattr(v, 'memory_usage'):
            mem = v.memory_usage(deep=True)
            if not np.isscalar(mem):
                mem = mem.sum()
            vars.setdefault(id(v),[mem]).append(k)
        # work around for a bug
        elif isinstance(v,pd.Panel):
            v = v.values
        vars.setdefault(id(v),[sys.getsizeof(v)]).append(k)
    total = 0
    for k,(value,*names) in vars.items():
        if value>1e6:
            print(names,"%.3fMB"%(value/1e6))
        total += value
    print("%.3fMB"%(total/1e6))

In [14]:
import time
graph = {}
revgraph = {}
ind = 0
start_time = time.clock()
for myline in mydata:
    entries = myline.split()
    tail = int(entries[0])
    if tail >= 1 and tail <= num_vertices:
        tail = str(tail)
    else:
        print(entries)
        break
    head = int(entries[1])
    if head >= 1 and head <= num_vertices:
        head = str(head)
    else:
        print(entries)
        break
    if(tail in graph):
        graph[tail].append(head)
    else:
        graph[tail]=[head]    
    if head in revgraph:
        revgraph[head].append(tail)
    else:
        revgraph[head] = [tail]    
    ind += 1
print(ind,len(graph),len(revgraph),time.clock()-start_time)
show_mem_usage()
global cur_time
fin_time = {}

5105043 739454 714547 13.720555000000001
['mydata'] 42.915MB
['revgraph'] 50.332MB
['graph'] 50.332MB
['explored'] 50.332MB
['fin_time'] 50.332MB
244.250MB


In [15]:
explored = {}
for ii in range(1,num_vertices+1):
    explored[str(ii)]=False
show_mem_usage()
global rec_depth
rec_depth =0

['explored'] 50.332MB
['revgraph'] 50.332MB
['graph'] 50.332MB
['mydata'] 42.915MB
193.918MB


In [16]:
def print_debug(node_id):
    global rec_depth
    print(rec_depth,node_id)

In [17]:
def dfs(graph1,node_id):
    global cur_time
    global rec_depth
    count = 0
    rec_depth += 1
    explored[node_id]=True
    count += 1
    if node_id in graph1:
        for myedge in graph1[node_id]:
            if  not explored[myedge]:
                count += dfs(graph1,myedge)
    cur_time += 1
    fin_time[str(cur_time)] = node_id
    rec_depth -= 1
    return count

In [18]:
cur_time = 0
start_time = time.clock()
for ii in range(1,num_vertices+1):
    if ii % 100000 == 0:
        print(ii,time.clock()-start_time)
    if not explored[str(ii)]:
        dfs(revgraph,str(ii))

10000 2.831970999999996
20000 2.851779999999998
30000 2.8715569999999957
40000 2.8889569999999978
50000 2.9043009999999967
60000 2.920294999999996
70000 2.9351549999999946
80000 2.9501649999999984
90000 2.964452999999999
100000 2.980415999999998
110000 2.994944999999994
120000 3.0079700000000003
130000 3.0213119999999947
140000 3.034842999999995
150000 3.0480349999999987
160000 3.061746999999997
170000 3.074645999999994
180000 3.087213999999996
190000 3.1004179999999977
200000 3.1132319999999964
210000 3.1275149999999954
220000 3.143566
230000 3.1564929999999976
240000 3.1910569999999936
250000 3.2034909999999996
260000 3.216321999999998
270000 3.228445999999998
280000 3.241726
290000 3.254179999999998
300000 3.2667829999999967
310000 3.279164999999999
320000 3.291395999999999
330000 3.3034339999999958
340000 3.315760999999995
350000 3.3273949999999957
360000 3.338963999999997
370000 3.3512909999999962
380000 3.3632519999999957
390000 3.375397999999997
400000 3.3871819999999957
410000 

In [19]:
explored = {}
largest_scc = [0,0,0,0,0]
for ii in range(1,num_vertices+1):
    explored[str(ii)]=False
for ii in range(num_vertices,0,-1):
    if not explored[fin_time[str(ii)]]:
        scc_size = dfs(graph,fin_time[str(ii)])
        if scc_size > largest_scc[0]:
            largest_scc[0] = scc_size
            largest_scc = sorted(largest_scc)
print(largest_scc)

[211, 313, 459, 968, 434821]
